In [22]:
# The following taken from 2021 notebook

# allows editing aoc_utils "live" without restarting kernel
# see https://ipython.org/ipython-doc/stable/config/extensions/autoreload.html
# and https://stackoverflow.com/a/17551284
%load_ext autoreload
%autoreload 2

# Add the aoc_utils path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import aoc_utils
get_input = aoc_utils.get_input
print = aoc_utils.debug_print

# Useful imports
import re
from collections import defaultdict, deque
import heapq
import functools
import queue
import itertools
import math
import random
from collections import Counter
import statistics
import parse
import operator
from functools import reduce

# aliases from utils
getnums = aoc_utils.getnums

# from norvig's pytudes
cat = ''.join

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
def parse(data,p2=False):
  G = defaultdict(int)
  ymax = len(data)
  xmax = len(data[0])
  for y,line in enumerate(data):
    for x,c in enumerate(line):
      if p2 and x == 2 and y == 2: continue
      pos = (x,y) if not p2 else (x,y,0)
      G[pos] = 1 if c == '#' else 0
  return xmax,ymax,G

def hash(g,xmax,ymax):
  v = 0
  idx = 0
  for y in range(xmax):
    for x in range(ymax):
      if g[(x,y)] == 1: v += 2**idx
      idx += 1
  return v

ex = """.....
.....
.....
#....
.#...""".split('\n')

xmax,ymax,g = parse(ex)
assert hash(g,xmax,ymax) == 2129920

In [50]:
def go():
  data = get_input(24,2019)
  xmax,ymax,G = parse(data)

  def tick(g):
    g2 = defaultdict(int)
    for pos in itertools.product(range(xmax),range(ymax)):
      adj = len([n for n in aoc_utils.neighbors4(pos) if g[n] == 1])
      v = g[pos]
      v2 = None
      if v == 1:
        v2 = 1 if adj == 1 else 0
      elif v == 0:
        v2 = 1 if adj in [1,2] else 0
      else:
        assert False
      g2[pos] = v2
    return g2

  seen = set([hash(G,xmax,ymax)])
  cnt = 0
  while cnt < 10000:
    cnt += 1
    G = tick(G)
    h = hash(G,xmax,ymax)
    if h in seen:
      return h
    seen.add(h)

assert go() == 25719471

In [59]:
def part2():
  data = get_input(24,2019)
  xmax,ymax,g = parse(data,p2=True)

  def n4(p):
    x,y,lvl = p
    assert not (x==2 and y==2)
    N = []
    E = []
    S = []
    W = []
    if y == 0:
      N = [(2,1,lvl+1)]
    elif y == 3 and x == 2:
      N = [
        (0,4,lvl-1),
        (1,4,lvl-1),
        (2,4,lvl-1),
        (3,4,lvl-1),
        (4,4,lvl-1),
      ]
    else:
      N = [(x,y-1,lvl)]

    if x == 4:
      E = [(3,2,lvl+1)]
    elif x == 1 and y == 2:
      E = [
        (0,0,lvl-1),
        (0,1,lvl-1),
        (0,2,lvl-1),
        (0,3,lvl-1),
        (0,4,lvl-1),
      ]
    else:
      E = [(x+1,y,lvl)]

    if y == 4:
      S = [(2,3,lvl+1)]
    elif y == 1 and x == 2:
      S = [
        (0,0,lvl-1),
        (1,0,lvl-1),
        (2,0,lvl-1),
        (3,0,lvl-1),
        (4,0,lvl-1),
      ]
    else:
      S = [(x,y+1,lvl)]

    if x == 0:
      W = [(1,2,lvl+1)]
    elif x == 3 and y == 2:
      W = [
        (4,0,lvl-1),
        (4,1,lvl-1),
        (4,2,lvl-1),
        (4,3,lvl-1),
        (4,4,lvl-1),
      ]
    else:
      W = [(x-1,y,lvl)]

    ns = set()
    for p in N:
      assert p not in ns
      ns.add(p)
    for p in E:
      assert p not in ns
      ns.add(p)
    for p in S:
      assert p not in ns
      ns.add(p)
    for p in W:
      assert p not in ns
      ns.add(p)
    return list(ns)

  def tick(g):
    # gcopy = g.copy()
    g2 = defaultdict(int)
    points = set()
    for p in g.keys():
      for n in n4(p):
        points.add(n)
    for p in points:
      ns = n4(p)
      cnt = len([n for n in ns if g[n] == 1])
      if (g[p] == 1 and cnt == 1) or (g[p] == 0 and cnt in [1,2]):
        g2[p] = 1
      else:
        g2[p] = 0
    return g2

  for _ in range(200):
    g = tick(g)
  return len([p for p in g.keys() if g[p] == 1])

assert part2() == 1916



1916